# Skin Cancer MNIST: HAM10000--Using Keras CNN

This is a multi-class classification problem. To faster implement the solution, I used the pre-trained MobileNet structure in Keras. Since the dataset is very imbalanced, I used data augmentation to make the size of each class approximately equal.

In [3]:
import os
import shutil

import cv2
import gc
import keras
import numpy as np
import pandas as pd
from keras.applications.mobilenet import MobileNet
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import (BatchNormalization, Dense, Dropout, Flatten)
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import ResNet50

# 1 Data Exploration

First check what columns are in the metadata.

In [4]:
metadata = pd.read_csv("/kaggle/input/HAM10000_metadata.csv")
metadata.head()

Now check the proportion of each label.

In [5]:
metadata["dx"].value_counts() / metadata.shape[0]

The dataset is very uneven. Thus, we need to augment the data later.

Check the image format.

In [6]:
image_sample = cv2.imread("/kaggle/input/ham10000_images_part_1/ISIC_0027269.jpg")
print(image_sample.shape)

It was mentioned that there may be multiple images for one lesion. Thus, we need to use solely those lesions that have no duplicates (i.e. no other images of the same lesion) as the validation set and the test set. Now we add one more column to the table to mark if the lesion have duplicate images.

In [8]:
lesion_id_cnt = metadata["lesion_id"].value_counts()
def check_duplicates(id):
    
    if lesion_id_cnt[id] > 1:
        return True
    else:
        return False

metadata["has_duplicate"] = metadata["lesion_id"].map(check_duplicates)

In [9]:
metadata["has_duplicate"].value_counts()

Images are stored in 2 different folders. Thus, we need to mark which folder each specific image is in.

In [10]:
image_folder_1 = "/kaggle/input/ham10000_images_part_1"
image_folder_2 = "/kaggle/input/ham10000_images_part_2"
metadata["folder"] = 0
metadata.set_index("image_id", drop=False, inplace=True)

for image in os.listdir(image_folder_1):
    image_id = image.split(".")[0]
    metadata.loc[image_id, "folder"] = "1"

for image in os.listdir(image_folder_2):
    image_id = image.split(".")[0]
    metadata.loc[image_id, "folder"] = "2"

Now let's take a look at the data.

In [11]:
metadata.head()

# 2 Split the Data

Now I need to split the data into training/validation/test datasets. I split the data in a 80%-10%-10% fashion.

In [12]:
data_train_no_dup, data_val = train_test_split(metadata[metadata["has_duplicate"] == False], test_size=0.36, stratify=metadata[metadata["has_duplicate"] == False]["dx"]) # 36% of the data with no duplicates is roughly 20% of the total
data_train = pd.concat((data_train_no_dup, metadata[metadata["has_duplicate"] == True]), axis=0)
data_val, data_test = train_test_split(data_val, test_size=0.5, stratify=data_val["dx"])
print("Train: " + str(data_train.shape[0] / metadata.shape[0]))
print("Validation: " + str(data_val.shape[0] / metadata.shape[0]))
print("Test: " + str(data_test.shape[0] / metadata.shape[0]))
val_len = data_val.shape[0]
test_len = data_test.shape[0]

Now make new directories for train/val/test images.

In [16]:
base_directory = "base_directory"
os.mkdir(base_directory)

train_dir = os.path.join(base_directory, "image_train")
os.mkdir(train_dir)

val_dir = os.path.join(base_directory, "image_val")
os.mkdir(val_dir)

test_dir = os.path.join(base_directory, "image_test")
os.mkdir(test_dir)

labels = list(metadata["dx"].unique())

for label in labels:
    label_path_train = os.path.join(train_dir, label)
    os.mkdir(label_path_train)
    label_path_val = os.path.join(val_dir, label)
    os.mkdir(label_path_val)
    label_path_test = os.path.join(test_dir, label)
    os.mkdir(label_path_test)

Copy the images to the new directory.

In [39]:
image_dir = "/kaggle/input/ham10000_images_part_"

for i in range(data_train.shape[0]):
    image_name = data_train["image_id"][i] + ".jpg"
    src_dir = os.path.join(image_dir + data_train["folder"][i], image_name)
    dst_dir = os.path.join(train_dir, data_train["dx"][i], image_name)
    shutil.copyfile(src_dir, dst_dir)

for i in range(data_val.shape[0]):
    image_name = data_val["image_id"][i] + ".jpg"
    src_dir = os.path.join(image_dir + data_val["folder"][i], image_name)
    dst_dir = os.path.join(val_dir, data_val["dx"][i], image_name)
    shutil.copyfile(src_dir, dst_dir)
    
for i in range(data_test.shape[0]):
    image_name = data_test["image_id"][i] + ".jpg"
    src_dir = os.path.join(image_dir + data_test["folder"][i], image_name)
    dst_dir = os.path.join(test_dir, data_test["dx"][i], image_name)
    shutil.copyfile(src_dir, dst_dir)

Now check the proportions of each label in each dataset.

In [41]:
for label in labels:
    print(label + " train: " + str(len(os.listdir(os.path.join(train_dir, label)))))
print("\n")
for label in labels:
    print(label + " val: " + str(len(os.listdir(os.path.join(val_dir, label)))))
print("\n")
for label in labels:
    print(label + " val: " + str(len(os.listdir(os.path.join(test_dir, label)))))

Delete the redundant data and collect the RAM.

In [ ]:
del data_train_no_dup, metadata
gc.collect()

# 3 Augment the Data

Now I need to augment the data and save the augmented images in each folder.

In [ ]:
data_gen_param = {
    "rotation_range": 180,
    "width_shift_range": 0.1,
    "height_shift_range": 0.1,
    "zoom_range": 0.1,
    "horizontal_flip": True,
    "vertical_flip": True
}
data_generator = ImageDataGenerator(**data_gen_param)
num_images_each_label = 1500

aug_dir = os.path.join(base_directory, "aug_dir")
os.mkdir(aug_dir)

for label in labels:
    
    img_dir = os.path.join(aug_dir, "aug_img")
    os.mkdir(img_dir)
    
    src_dir_label = os.path.join(train_dir, label)
    for image_name in os.listdir(src_dir_label):
        shutil.copy(os.path.join(src_dir_label, image_name), os.path.join(img_dir, image_name))
    
    batch_size = 32
    data_flow_param = {
        "directory": aug_dir,
        "color_mode": "rgb",
        "batch_size": batch_size,
        "shuffle": True,
        "save_to_dir": os.path.join(train_dir, label),
        "save_format": "jpg"
    }
    aug_data_gen = data_generator.flow_from_directory(**data_flow_param)
    
    num_img_aug = num_images_each_label - len(os.listdir(os.path.join(train_dir, label)))
    num_batch = int(num_img_aug / batch_size)
    
    for i in range(0, num_batch):
        next(aug_data_gen)
    
    shutil.rmtree(img_dir)

Now check if the data is balanced.

In [ ]:
for label in labels:
    print(label + " train: " + str(len(os.listdir(os.path.join(train_dir, label)))))
print("\n")
for label in labels:
    print(label + " val: " + str(len(os.listdir(os.path.join(val_dir, label)))))

# 4 Set Up the Generator

I used generator to train the model since it saves up RAM.

In [ ]:
IMAGE_SHAPE = (450, 600, 3) # original size 
data_gen_param = {
    "samplewise_center": True,
    "samplewise_std_normalization": True,
    "rotation_range": 180,
    "width_shift_range": 0.1,
    "height_shift_range": 0.1,
    "zoom_range": 0.1,
    "horizontal_flip": True,
    "vertical_flip": True,
    "rescale": 1.0 / 255
}
data_generator = ImageDataGenerator(**data_gen_param)

train_flow_param = {
    "directory": train_dir,
    "batch_size": batch_size,
    "target_size": IMAGE_SHAPE[:2],
    "shuffle": True
}
train_flow = data_generator.flow_from_directory(**train_flow_param)

val_flow_param = {
    "directory": val_dir,
    "batch_size": batch_size,
    "target_size": IMAGE_SHAPE[:2],
    "shuffle": False
}
val_flow = data_generator.flow_from_directory(**val_flow_param)

test_flow_param = {
    "directory": test_dir,
    "batch_size": 1,
    "target_size": IMAGE_SHAPE[:2],
    "shuffle": False
}
test_flow = data_generator.flow_from_directory(**test_flow_param)

# 5 Train the Model

Now set up all the hyper-parameters and train the model. Here I used the pre-trained MobileNet that have been trained on ImageNet classification task as the CNN structure and added a 7-unit softmax activation at the end of the network. Here, I also used top-2 and top-3 accuracy metrics.

In [ ]:
# dropout_dense = 0.1

# vgg16 = keras.applications.vgg16.VGG16(include_top=False, input_shape=IMAGE_SHAPE, pooling="max")

# model = Sequential()
# model.add(vgg16)
# model.add(Dropout(dropout_dense))
# model.add(BatchNormalization())
# model.add(Dense(256, activation="relu"))
# model.add(Dropout(dropout_dense))
# model.add(BatchNormalization())
# model.add(Dense(256, activation="relu"))
# model.add(Dense(7, activation="softmax"))

# mobilenet_model = MobileNet(input_shape=IMAGE_SHAPE, include_top=False, pooling="max")

"""
model = Sequential()
model.add(mobilenet_model)
model.add(Dropout(dropout_dense))
model.add(BatchNormalization())
model.add(Dense(256, activation="relu"))
model.add(Dropout(dropout_dense))
model.add(BatchNormalization())
model.add(Dense(7, activation="softmax"))

def top_2_acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

def top_3_acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

model.compile(Adam(0.01), loss="categorical_crossentropy", metrics=[categorical_accuracy, top_2_acc, top_3_acc])

In [ ]:
model_32 = ResNet50(include_top=True, weights=None, input_shape=(32, 32, 3), pooling=max, classes=7)
model_64 = ResNet50(include_top=True, weights=None, input_shape=(64, 64, 3), pooling=max, classes=7)
model_128 = ResNet50(include_top=True, weights=None, input_shape=(128, 128, 3), pooling=max, classes=7)
model_256 = ResNet50(include_top=True, weights=None, input_shape=(256, 256, 3), pooling=max, classes=7)

In [ ]:
model_32.summary()
model_64.summary()
model_128.summary()
model_256.summary()

In [ ]:
model_32.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])
model_64.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])
model_128.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])
model_256.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])

# Training

In [ ]:
batch_size = 16 
epochs = 50

history = model_32.fit(
    x_train, y_train,
    epochs=epochs,
    batch_size = batch_size,
    validation_data=(x_test, y_test)
    )

score = model.evaluate( x_test, y_test)
print('Test accuracy:', score[1])

In [ ]:
batch_size = 16 
epochs = 50

history = model_64.fit(
    x_train, y_train,
    epochs=epochs,
    batch_size = batch_size,
    validation_data=(x_test, y_test)
    )

score = model.evaluate( x_test, y_test)
print('Test accuracy:', score[1])

In [ ]:
batch_size = 16 
epochs = 50

history = model_128.fit(
    x_train, y_train,
    epochs=epochs,
    batch_size = batch_size,
    validation_data=(x_test, y_test)
    )

score = model.evaluate( x_test, y_test)
print('Test accuracy:', score[1])

In [ ]:
batch_size = 16 
epochs = 50

history = model_256.fit(
    x_train, y_train,
    epochs=epochs,
    batch_size = batch_size,
    validation_data=(x_test, y_test)
    )

score = model.evaluate( x_test, y_test)
print('Test accuracy:', score[1])

I added checkpoint to track the performance of the model along each epoch. Also, I used learning rate decay and early stopping the get better convergence and prevent overfitting.

In [ ]:
filepath = "model.h5"

checkpoint_param = {
    "filepath": filepath,
    "monitor": "val_categorical_accuracy",
    "verbose": 1,
    "save_best_only": True,
    "mode": "max"
}
checkpoint = ModelCheckpoint(**checkpoint_param)

lr_decay_params = {
    "monitor": "val_loss",
    "factor": 0.5,
    "patience": 2,
    "min_lr": 1e-5
}
lr_decay = ReduceLROnPlateau(**lr_decay_params)

early_stopping = EarlyStopping(monitor="val_loss", patience=4, verbose=1)

Now train the model.

In [ ]:
fit_params = {
    "generator": train_flow,
    "steps_per_epoch": data_train.shape[0] // batch_size,
    "epochs": 15,
    "verbose": 1,
    "validation_data": val_flow,
    "validation_steps": data_val.shape[0] // batch_size,
    "callbacks": [checkpoint, lr_decay, early_stopping]
}
print("Training the model...")

history = model.fit_generator(**fit_params)
print("Done!")

# 6 Evaluate the Model

See the accuracies and F1 scores on validation and test sets.

In [ ]:
_, val_acc, val_top_2_acc, val_top_3_acc = model.evaluate_generator(val_flow, steps=len(val_flow))
y_val_true = val_flow.classes
y_val_pred = np.argmax(model.predict_generator(val_flow, steps=len(val_flow)), axis=1)
val_f1_score = f1_score(y_val_true, y_val_pred, average="micro")

print("Validation accuracy: {:.4f}".format(val_acc))
print("Validation top-2 accuracy: {:.4f}".format(val_top_2_acc))
print("Validation top-3 accuracy: {:.4f}".format(val_top_3_acc))
print("Validation F1 score: {:.4f}".format(val_f1_score))

In [ ]:
_, test_acc, test_top_2_acc, test_top_3_acc = model.evaluate_generator(test_flow, steps=len(test_flow))
y_test_true = test_flow.classes
y_test_pred = np.argmax(model.predict_generator(test_flow, steps=len(test_flow)), axis=1)
test_f1_score = f1_score(y_test_true, y_test_pred, average="micro")

print("Test accuracy: {:.4f}".format(test_acc))
print("Test top-2 accuracy: {:.4f}".format(test_top_2_acc))
print("Test top-3 accuracy: {:.4f}".format(test_top_3_acc))
print("Test F1 score: {:.4f}".format(test_f1_score))

Track the performance on each epoch.

In [ ]:
loss_train = history.history["loss"]
acc_train = history.history["categorical_accuracy"]
loss_val = history.history["val_loss"]
acc_val = history.history["val_categorical_accuracy"]
epochs = np.arange(1, len(loss_train) + 1)

In [ ]:
plt.plot(epochs, acc_train, "bo", label="Training acc")
plt.plot(epochs, acc_val, "b", label="Validation acc")
plt.title("Accuracy")
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, loss_train, "bo", label="Training loss")
plt.plot(epochs, loss_val, "b", label="Validation loss")
plt.title("Losses")
plt.legend()
plt.show()

Plot the confusion matrix to see where the model made mistakes.

In [ ]:
conf_mat = confusion_matrix(y_test_true, y_test_pred)
plt.imshow(conf_mat, interpolation="nearest", cmap=plt.cm.Blues)
plt.title("Confusion matrix")
plt.colorbar()
tick_marks = np.arange(len(labels))
plt.xticks(tick_marks, labels, rotation=45)
plt.yticks(tick_marks, labels)
plt.ylabel("y_true")
plt.xlabel("y_pred")
plt.tight_layout()

Delete the image folder so that it won't output that many files when committed.

In [ ]:
shutil.rmtree(base_dir)

# 7 Future Improvement

- Use stacked voting model to solve the data imbalance;
- Larger augmented datasets;
- Testing-time augmentation.